In [4]:
!ls

__init__.py  logs    notebooks	  pre-processing  utils
data	     models  poetry.lock  pyproject.toml


In [2]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import pandas as pd

df = pd.read_csv('data/rated_2014-01_initial_cleaning.csv')
df = df[df['event'] == 'classical']
df = df[df.columns[1:]]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246495 entries, 2 to 697466
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   white_elo     246495 non-null  float64
 1   black_elo     246495 non-null  float64
 2   time_control  246495 non-null  object 
 3   result        246495 non-null  object 
 4   termination   246495 non-null  object 
 5   moves         246495 non-null  object 
dtypes: float64(2), object(4)
memory usage: 13.2+ MB


In [6]:
import fasttext
embeddings_model = fasttext.load_model('models/chess2vec-3ws-32dim.bin') 

In [7]:
from utils.data import ChessDataset
from torch.utils.data import DataLoader
dataset = ChessDataset(df, embeddings_model=embeddings_model, device=device)

In [8]:
from torch.utils.data import random_split
from utils.data import collate_data

train_dataset, valid_dataset = random_split(dataset, [0.8, 0.2])
train_loader = DataLoader(train_dataset, batch_size=64, collate_fn=collate_data, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, collate_fn=collate_data, shuffle=False)

In [55]:
from torch.optim import Adam
from torch.nn import MSELoss
from utils.models import ELOPredictor
model = ELOPredictor()
optimizer = Adam(model.parameters())
loss = MSELoss()

model = torch.compile(model.to(device))

epochs = 5
for epoch in range(epochs):
    print(f'Epoch: {epoch+1}')
    train(model, train_loader, optimizer, loss, device)
    validate(model, valid_dataset, loss)


Epoch: 1


KeyboardInterrupt: 